# COVID-19 mRNA Vaccine Degradation Prediction


**Context: The most promising COVID-19 vaccine candidates use messenger RNA molecules (mRNA)** to help the patient develop immunity. Unfortunately, mRNA molecules are extremely fragile, and in certain circumstances spontaneously degrade. Current best scientific understanding lacks a means to analyze mRNA molecule candidates for areas of likely degradation.

**Goal: Predict degradation rates for each part of an RNA molecule.**

![mrna diagram](https://upload.wikimedia.org/wikipedia/commons/f/fb/MRNA-interaction.png)

In [ ]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

## Automated hyper-parameter optimization

In [ ]:
# Hyper-parameters
LR = 1e-3
EPOCHS = 4
BATCH_SIZE = 64
EMBED_DIM = 100
HIDDEN_DIM = 128
DROPOUT = .5
SP_DROPOUT = .3
TRAIN_SEQUENCE_LENGTH = 107

## Load data

In [ ]:
train_df = pd.read_json("data/train.json", lines=True)
test_df = pd.read_json("data/test.json", lines=True)

In [ ]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")

## Preprocess data

In [ ]:
symbols = "().ACGUBEHIMSX"
feat_cols = ["sequence", "structure", "predicted_loop_type"]
target_cols = ["reactivity", "deg_Mg_pH10", "deg_Mg_50C", "deg_pH10", "deg_50C"]
error_cols = ["reactivity_error", "deg_error_Mg_pH10", "deg_error_Mg_50C", "deg_error_pH10", "deg_error_50C"]

In [ ]:
tokenizer = Tokenizer(char_level=True, filters="")
tokenizer.fit_on_texts(symbols)

In [ ]:
# get the number of elements in the vocabulary
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
def process_features(example):
    sequence_sentences = example[0]
    structure_sentences = example[1]
    loop_sentences = example[2]
    
    # transform character sequences into number sequences
    sequence_tokens = np.array(
        tokenizer.texts_to_sequences(sequence_sentences)
    )
    structure_tokens = np.array(
        tokenizer.texts_to_sequences(structure_sentences)
    )
    loop_tokens = np.array(
        tokenizer.texts_to_sequences(loop_sentences)
    )
    
    # concatenate the tokenized sequences
    sequences = np.stack(
        (sequence_tokens, structure_tokens, loop_tokens),
        axis=1
    )
    sequences = np.transpose(sequences, (2, 0, 1))
    
    prepared = sequences.tolist()
    
    return prepared[0]

In [ ]:
def process_labels(df):
    df = df.copy()
    
    labels = np.array(df[target_cols].values.tolist())
    labels = np.transpose(labels, (0, 2, 1))
    
    return labels

In [ ]:
public_test_df = test_df.query("seq_length == 107")
private_test_df = test_df.query("seq_length == 130")

In [ ]:
x_train = [process_features(row.tolist()) for _, row in train_df[feat_cols].iterrows()]
y_train = process_labels(train_df)

unprocessed_x_public_test = [row.tolist() for _, row in public_test_df[feat_cols].iterrows()]
unprocessed_x_private_test = [row.tolist() for _, row in private_test_df[feat_cols].iterrows()]

# Define and train the model

In [ ]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
         tf.keras.layers.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def build_model(vocab_size, seq_length=TRAIN_SEQUENCE_LENGTH, pred_len=68,
                embed_dim=EMBED_DIM,
                hidden_dim=HIDDEN_DIM, dropout=DROPOUT, sp_dropout=SP_DROPOUT):
    inputs = tf.keras.layers.Input(shape=(seq_length, 3))

    embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    
    hidden = tf.keras.layers.SpatialDropout1D(sp_dropout)(reshaped)
    
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation="linear")(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model

In [ ]:
model = build_model(vocab_size)

In [ ]:
class MeanColumnwiseRMSE(tf.keras.losses.Loss):
    def __init__(self, name='MeanColumnwiseRMSE'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [ ]:
model.compile(tf.optimizers.Adam(learning_rate=LR), loss=MeanColumnwiseRMSE())

In [ ]:
history = model.fit(np.array(x_train), np.array(y_train), 
                    validation_split=.1, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
validation_loss = history.history.get("val_loss")[0]

## Evaluate the model

In [ ]:
model_public = build_model(vocab_size, seq_length=107, pred_len=107)
model_private = build_model(vocab_size, seq_length=130, pred_len=130)

model_public.set_weights(model.get_weights())
model_private.set_weights(model.get_weights())

In [ ]:
public_preds = model_public.predict(np.array([process_features(x) for x in unprocessed_x_public_test]))
private_preds = model_private.predict(np.array([process_features(x) for x in unprocessed_x_private_test]))

In [ ]:
print(validation_loss)